[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LayerDiffuse-jupyter/blob/main/LayerDiffuse_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b study https://github.com/camenduru/ComfyUI /content/totoro
!pip install -q torchsde diffusers accelerate einops xformers==0.0.25
%cd /content/totoro

from typing import Optional
import os, torch
from urllib.parse import urlparse
import numpy as np

def load_file_from_url(url: str, *, model_dir: str, progress: bool = True, file_name: Optional[str] = None,) -> str:
    os.makedirs(model_dir, exist_ok=True)
    if not file_name:
        parts = urlparse(url)
        file_name = os.path.basename(parts.path)
    cached_file = os.path.abspath(os.path.join(model_dir, file_name))
    if not os.path.exists(cached_file):
        print(f'Downloading: "{url}" to {cached_file}\n')
        from torch.hub import download_url_to_file
        download_url_to_file(url, cached_file, progress=progress)
    return cached_file

ckpt_path = load_file_from_url(url="https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors", model_dir="/content/model", file_name="Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors")

import totoro.sd, torch
ckpt_path = "/content/model/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors"
with torch.no_grad():
  model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config(ckpt_path, output_vae=True, output_clip=True, embedding_directory=None)

from rng_noise_generator import ImageRNGNoise
def generate_latent(width, height, seed, subseed, subseed_strength, seed_resize_from_h=None, seed_resize_from_w=None):
    shape = [4, height // 8, width // 8]
    rng = ImageRNGNoise(shape=shape, seeds=[seed], subseeds=[subseed], subseed_strength=subseed_strength, seed_resize_from_h=seed_resize_from_h, seed_resize_from_w=seed_resize_from_w)
    noise = rng.next()
    return {"samples": noise}
latent = generate_latent(1024, 1024, 1, 1, 0, 1024, 1024)

def common_ksampler_with_custom_noise(model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent,
                                      denoise=1.0, disable_noise=False, start_step=None, last_step=None,
                                      force_full_denoise=False, noise=None):
    latent_image = latent["samples"]
    if noise is not None:
        rng_noise = noise.next().detach().cpu()
        noise = rng_noise.clone()
    else:
        if disable_noise:
            noise = torch.zeros(latent_image.size(), dtype=latent_image.dtype, layout=latent_image.layout, device="cpu")
        else:
            batch_inds = latent["batch_index"] if "batch_index" in latent else None
            from totoro.sample import prepare_noise
            noise = prepare_noise(latent_image, seed, batch_inds)

    noise_mask = None
    if "noise_mask" in latent:
        noise_mask = latent["noise_mask"]

    from totoro.sample import sample as sample_k

    samples = sample_k(model, noise, steps, cfg, sampler_name, scheduler, positive, negative, latent_image,
                       denoise=denoise, disable_noise=disable_noise, start_step=start_step,
                       last_step=last_step,
                       force_full_denoise=force_full_denoise, noise_mask=noise_mask, callback=None,
                       disable_pbar=False, seed=seed)
    out = latent.copy()
    out["samples"] = samples

    return (out, )

def get_conds(prompt):
    with torch.inference_mode():
        clip_skip = -2
        if clip_skip != clip_skip or clip.layer_idx != clip_skip:
            clip.layer_idx = clip_skip
            clip.clip_layer(clip_skip)
            clip_skip = clip_skip
        tokens = clip.tokenize(prompt)
        cond, pooled = clip.encode_from_tokens(tokens, return_pooled=True)
        return [[cond, {"pooled_output": pooled}]]



# from PIL import Image

# cond = get_conds("duck")
# n_cond = get_conds("")
# sample = common_ksampler_with_custom_noise(model=model_patcher, seed=0, steps=20, cfg=8.0, sampler_name="euler", scheduler="normal", positive=cond, negative=n_cond,
#                                            latent=latent, denoise=1, disable_noise=False, start_step=0, last_step=20, force_full_denoise=True, noise=None)
# def decode_sample(sample):
#     with torch.inference_mode():
#         sample = sample.to(torch.float32)
#         vae.first_stage_model.cuda()
#         decoded = vae.decode_tiled(sample).detach()
#     return decoded
# decoded = decode_sample(sample[0]["samples"])
# np_array = np.clip(255. * decoded.cpu().numpy(), 0, 255).astype(np.uint8)[0]
# image = Image.fromarray(np_array)
# image = image.convert("RGBA")
# # sample[0]["samples"]
# image

In [ ]:
%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI-layerdiffuse /content/layerdiffuse
%cd /content/layerdiffuse

import os
from enum import Enum
import torch
import functools
import copy
from typing import Optional, List
from dataclasses import dataclass

# import folder_paths
import totoro.model_management
import totoro.model_base
import totoro.supported_models
import totoro.supported_models_base
from totoro.model_patcher import ModelPatcher
# from folder_paths import get_folder_paths
from totoro.utils import load_torch_file
# from totoro.nodes_compositing import JoinImageWithAlpha
from totoro.conds import CONDRegular
from lib_layerdiffusion.utils import (
    load_file_from_url,
    to_lora_patch_dict,
)
from lib_layerdiffusion.models import TransparentVAEDecoder
from lib_layerdiffusion.attention_sharing import AttentionSharingPatcher
from lib_layerdiffusion.enums import StableDiffusionVersion